In [ ]:
Have you replaced your model folder with the correct model

In [6]:
import pandas as pd
import duckdb as db
import streamlit as st
from bertopic import BERTopic
name = 'perplexity'
input_csv = f'../../discord_mining/text_processing/results/{name}/{name}_results.csv'
model_folder = f'../../discord_mining/text_processing/model/{name}'

df_raw = pd.read_csv(input_csv, usecols=['Author', 
                                           'Date',
                                           'Content',
                                           'protocol',
                                           'topic_cluster', 
                                           'topic_count',
                                           'Name',
                                           'Representation', 
                                           'Representative_Docs',
                                           'sentiment_score',
                                           'is_political', 
                                           'is_product_complaint',
                                           'is_product_feedback', 
                                           'is_user_painpoint', 
                                           'is_user_request',
                                           'is_noise',
                                           'topic_name'])


df_raw = db.sql("""
    select
        Author as author
        , Date as date
        , Content as content
        , protocol
        , topic_cluster
        , sentiment_score
        , topic_count
        , Name as name
        , Representation as representation
        , Representative_Docs as representative_docs
        , is_product_complaint
        , is_product_feedback
        , is_user_painpoint
        , is_user_request
        , is_noise
        , topic_name
    from df_raw
    where is_political = 0
      and topic_cluster != -1
""").df()


fact_posts = df_raw.loc[:,['author','date','content', 'protocol','topic_cluster','sentiment_score']].drop_duplicates()
dim_topic = df_raw.loc[:,[ 'topic_cluster', 
                            'topic_count',
                            'name',
                            'representation', 
                            'representative_docs',
                            'is_user_painpoint',
                            'is_product_feedback', 
                            'is_user_request',
                            'is_noise',
                            'topic_name'
                         ]].drop_duplicates()

# topic_model = BERTopic.load("./model")

fact_posts.sample(10).to_csv('fact_posts_sample.csv',index=False)
dim_topic.sample(10).to_csv('dim_topic_sample.csv',index=False)

fact_posts.to_csv('fact_posts.csv',index=False)
dim_topic.to_csv('dim_topic.csv',index=False)

